In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import rtbm.layers as layers
import rtbm.model as mdl

import warnings
warnings.filterwarnings('ignore')

from rtbm.costfunctions import mse
from rtbm import minimizer

from rtbm.mathtools import theta_1d
from riemann_theta.riemann_theta import RiemannTheta

# Derivatives test

In [3]:
Q=np.zeros((1,1), dtype=complex)
V=np.zeros((1,1), dtype=complex)
V[0,0]=0.1*2j*np.pi
Q[0,0]=0.01*2*np.pi

print(theta_1d(V,Q,3))

print(RiemannTheta(V/(2j*np.pi),1j*Q/(2*np.pi), derivs=[1,1,1]))

(-56011.7031448+1.67320075235e-12j)
(-56011.703144+8.08992919968e-13j)


# Linear layer test

In [4]:
inp = np.zeros((2,5))
inp[0,0] = 0.3
inp[1,0] = 0.5

L = layers.Linear(2,2)
L.feedin(inp,True)

array([[-0.45864231, -0.22072123, -0.22072123, -0.22072123, -0.22072123],
       [-0.06186401,  0.21189042,  0.21189042,  0.21189042,  0.21189042]])

# Layer test

In [ ]:
M = mdl.Model()
M.add(layers.ThetaUnitLayer(1,3))
M.add(layers.ThetaUnitLayer(3,1))

In [ ]:
def func(x):
    return np.sin(x)

X = np.linspace(0, 10, 5)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None]).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
minim = minimizer.CMA(True)
minim.train(mse, M, X, Y, tolfun=1e-3)

In [ ]:
npoints = 5
test_X = (np.linspace(0, 10, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.legend()

# Misc tests

In [ ]:
L = layers.MaxPosLayer(3,1)

In [ ]:
L.feedin(np.array([Y,2*Y,1*Y]).reshape(3,5))

In [ ]:
np.array([Y,2*Y,Y]).reshape(3,5)

In [ ]:
np.empty(0)

In [ ]:
derivative_1d_theta_phaseI([0.1],[1j],0)